# Python template

## Load libraries

In [ ]:
%pylab inline

import os
import _iso
import numpy as np
import numpy.ma as ma
from mpl_toolkits.basemap import Basemap, shiftgrid
from matplotlib.mlab import griddata
import matplotlib.colors as colors
from scipy.signal import medfilt2d
import netCDF4
import scipy.interpolate
from pandas import Series, DataFrame
import pandas as pd

import pyroms
import pyroms_toolbox
from bathy_smoother import *

os.environ["PYTHONPATH"] = "/Users/misumi/python:/usr/local/lib/python2.7/site-packages"
os.environ["PYROMS"] = "$PYTHONPATH/pyroms"
os.environ["PYROMS_GRIDID_FILE"] = "/Users/misumi/roms_data/gridid.txt"

## Set font

In [ ]:
plt.rc('font',family='Helvetica')
plt.rcParams["font.size"] = 20

---

## Def funcs

In [ ]:
def fill_mask_woa(var):
    a = var[:,:].copy()
    a = a.flatten()

    lat = grd_woa.hgrid.lat_rho[:]
    lon = grd_woa.hgrid.lon_rho[:]

    lat_1d = lat.flatten()
    lon_1d = lon.flatten()

    lat_1d = ma.masked_where(a.mask,lat_1d)
    lon_1d = ma.masked_where(a.mask,lon_1d)

    a = a.compressed()
    lat_1d = lat_1d.compressed()
    lon_1d = lon_1d.compressed()

    pts = np.zeros((lat_1d.size,2))

    pts[:,0] = lon_1d[:]
    pts[:,1] = lat_1d[:]

    b = scipy.interpolate.griddata(pts,a,(lon,lat),method='nearest')
    
    return b

---

## I/O 

### Write netCDF

In [ ]:
dst_nc00 = netCDF4.Dataset('../data/h025_l045_v001.clm-ecosys-woa.161206.nc','w')
dst_nc00.createDimension('ocean_time',12)
dst_nc00.createDimension('s_rho',nz)
dst_nc00.createDimension('eta_rho',ny)
dst_nc00.createDimension('xi_rho',nx)

fv = -999.0
no3_o = dst_nc00.createVariable('NO3',dtype('float').char,('ocean_time','s_rho','eta_rho','xi_rho'),fill_value=fv)

no3_o[:] = nudge[:]

dst_nc00.close()

### Read netCDF

In [ ]:
src_nc00 = netCDF4.Dataset('../data/woa2013v2/woa13_all_no3.nc','r')
var = src_nc00.variables['no3'][:,::-1,:,:]
src_nc00.close()

### list of varnames

In [ ]:
for vname,vinfo in src_nc00.variables.iteritems():
    print vname, vinfo.shape

### Copy dimes and vars from an exisiting file

In [ ]:
src_nc01 = netCDF4.Dataset('../data/h025_l045_v001.ini.161108.nc','r')
dst_nc00 = netCDF4.Dataset('../data/test.nc','w')
vnames = src_nc01.variables.keys()
dnames = src_nc01.dimensions.keys()

for d in dnames:
    x = src_nc01.dimensions.get(d)
    dst_nc00.createDimension(x.name,x.size)
    
fv = -999.0
for v in vnames:
    x = src_nc01.variables[v]
    y = dst_nc00.createVariable(x.name,np.dtype('float').char,(x.dimensions),fill_value=fv)

dst_nc00.close()

### Glob file list

In [ ]:
import glob
glob.glob('test/*')

## Matplotlib

In [ ]:
fig0 = plt.figure(figsize=(7,7))
ax0 = fig0.add_subplot(111)
ax0.plot(f,-z,'o',label='f')
ax0.set_ylim(-5500,0)
ax0.set_xlim(0,4)
ax0.set_xlabel('f')
ax0.set_ylabel('depth')
ax0.legend(loc='lower right')
ax0.grid()
fig0.tight_layout()

In [ ]:
savefig('../figs/C_fesed_CESM-Std.png',dpi=200,bbox_inches='tight')

---

## Basemap

In [ ]:
fig0 = plt.figure(figsize=(20,15))

lon_mn = np.min(grd_h025.hgrid.lon_vert)
lon_mx = np.max(grd_h025.hgrid.lon_vert)
lat_mn = np.min(grd_h025.hgrid.lat_vert)
lat_mx = np.max(grd_h025.hgrid.lat_vert)

v_mn = 0.0
v_mx = 3.6

ax0 = fig0.add_subplot(111)
m0 = Basemap(projection='cyl',llcrnrlat=lat_mn,urcrnrlat=lat_mx,llcrnrlon=lon_mn,urcrnrlon=lon_mx,resolution='c')
x,y = m0(grd_h025.hgrid.lon_rho,grd_h025.hgrid.lat_rho)
im0 = m0.pcolormesh(x,y,poc_h025[0,:,:],vmin=v_mn,vmax=v_mx)
m0.drawcoastlines()
cb0 = m0.colorbar(im0,'bottom',size='5%',pad='2%')

plt.tight_layout()

### Plot POP data

In [ ]:
lon2d = np.where(np.greater_equal(lon2d,min(lon2d[:,0])),lon2d-360,lon2d)
ep_gx1v6_2=ma.concatenate((ep_gx1v6,ep_gx1v6),axis=2)
lon2d_2=ma.concatenate((lon2d,lon2d+360.),axis=1)
lat2d_2=ma.concatenate((lat2d,lat2d),axis=1)

In [ ]:
def plot2d_gx1v6(var):
    src_nc00=netCDF4.Dataset('/Users/misumi/pop_data/gx1v6.concat.nc','r')
    tlon=src_nc00.variables['tlon'][:]
    tlat=src_nc00.variables['tlat'][:]
    vlon=src_nc00.variables['vlon'][:]
    vlat=src_nc00.variables['vlat'][:]
    src_nc00.close()
    
    var2=ma.concatenate((var,var),axis=1)
    
    fig0 = plt.figure(figsize=(20,10))
    
    lon_mn = 0.0
    lon_mx = 360.0
    lat_mn = -90.0
    lat_mx = 90.0

    ax0 = fig0.add_subplot(111)
    m0 = Basemap(projection='cyl',llcrnrlat=lat_mn,urcrnrlat=lat_mx,llcrnrlon=lon_mn,urcrnrlon=lon_mx,resolution='c')
    x,y = m0(vlon,vlat)
    im0 = m0.pcolormesh(x,y,var2)
    m0.drawcoastlines()
    cb0 = m0.colorbar(im0)

    plt.tight_layout()

---

## Color contour

### Set contour levels

In [ ]:
levels = ([3.2,5.6,10.0,18.0,32.0,56.0,100.0,180.0,320.0,560.0,1000.0,1800.0,3200.0,5600.0,10000.0])
norm = BoundaryNorm(levels, ncolors=cm.N, clip=True)
im0 = m0.pcolormesh(x,y,cs137s[cnt,:,:], norm=norm,cmap=cm)

### Add mark

In [ ]:
xx, yy = m0(lon0,lat0)
m0.plot(xx,yy,'ko',ms=10)

### Add text

In [ ]:
tx,ty = m0(140.1,39.5)
ax0.text(tx,ty,'${}^{137}\mathrm{Cs_{wat}} \:\: \mathrm{Bq \: m^{-3}}$',va='top',ha='left',fontsize=32)

### Define colormap

In [ ]:
def generate_cmap(colors):
    values = range(len(colors))
    vmax = np.ceil(np.max(values))
    color_list = []
    for v, c in zip(values, colors):
        color_list.append( ( v/ vmax, c) )
    return LinearSegmentedColormap.from_list('custom_cmap', color_list)

#### Usage

In [ ]:
cm = generate_cmap(['#ffffff','#6385c5','#6bad40','#eee959','#c04d29'])

### Colorbar

In [ ]:
cax = fig0.add_axes([0.557, 0.52, 0.02, 0.3])
cbar = fig0.colorbar(im0, cax=cax, ticks=[3.2,10.0,32.0,100.0,320.0,1000.0,3200.0,10000.0])
cbar.ax.set_yticklabels([3.2,10,32,100,320,1000,3200,10000])

---

## Datetime

In [ ]:
date0 = datetime.datetime(2011,3,1)
for n in range(3,30):
    dd = date0 + datetime.timedelta(n)
    print dd.strftime('%Y/%m/%d')

---

## Pyroms

### Read grids

In [ ]:
grd_h025 = pyroms.grid.get_ROMS_grid('h025_l045_v001')

---

## f2py

### FORTRAN (calc_sub.f)

In [ ]:
      subroutine calc(b,a,n)
      implicit none
c
      real*8  b(n)
      real*8  a(n)
      integer n, i

Cf2py intent(out) b
Cf2py intent(in)  a
Cf2py intent(in)  n

      do i = 1, n
        b(i) = a(i) + 1.0
      end do
c
      return
      end

### python

In [ ]:
import os
import numpy as np
import numpy.ma as ma
import netCDF4
from pandas import Series, DataFrame
import pandas as pd

import calc_sub

a = np.random.rand(10000)

b = calc_sub.calc(a,10000)

for n in range(10000):
  print a[n], b[n]

---

## Format

In [ ]:
print "{0:<10}".format("Hello") #10文字幅。左寄せ
print "{0:^10}".format("Hello") #10文字幅。センタリング
print "{0:>10}".format("Hello") #10文字幅。右寄せ
print "{0:_>10}".format("Hello") # 詰め文字に_を指定。10文字幅。右寄せ
print "{0:0>6}".format(123) #前ゼロ埋め

In [ ]:
print "int: {0:d};  hex: {0:X};  oct: {0:o};  bin: {0:b}".format(42)

In [ ]:
print "{0:,d}".format(1234567)

In [ ]:
print "{0:.3f}".format(12.34)

In [ ]:
import datetime
d = datetime.datetime.now()
"{0:%Y-%m-%d %H:%M:%S}".format(d)

---

## Remapping

In [ ]:
pyroms.remapping.make_remap_grid_file(grd_h025)

In [ ]:
pyroms.remapping.compute_remap_weights(\
                                       grid1_file='../remap/remap_grid_poc_rho.nc',\
                                       grid2_file='../remap/remap_grid_h025_l045_v003_rho.nc',\
                                       interp_file1='../remap/remap_wgt_poc_to_h025_l045_v003_rho.nc',\
                                       interp_file2='N/A', \
                                       map1_name='poc to h025_l045_v002 bilinear', \
                                       map2_name='N/A', \
                                       num_maps='1',\
                                       map_method='bilinear',\
                                       luse_grid1_area='.false.',\
                                       luse_grid2_area='.false.',\
                                       normalize_opt='fracarea',\
                                       output_opt='scrip',\
                                       restrict_type='latitude',\
                                       num_srch_bins='90',\
                                       grid1_periodic='.false.',\
                                       grid2_periodic='.false' )

In [ ]:
frac = pyroms.remapping.remap(topo_l[2100:4800,3000:9000],'../remap/remap_wgt_etopo2v2c_to_h025_l045_v002_rho.nc')

---

## Nearest grid indices in lat lon

In [ ]:
def arglatlon(latP,lonP,lat2d,lon2d):
    ny,nx=lat2d.shape
    lon0=np.zeros((ny,nx))+lonP
    lat0=np.zeros((ny,nx))+latP
    dlon=np.abs(lon2d-lon0)
    dlat=np.abs(lat2d-lat0)
    d=np.sqrt(dlon**2+dlat**2)
    ind1d=np.argmin(d)
    jind=ind1d/nx
    iind=ind1d%nx
    a=np.array([jind,iind])
    return a

### example

In [ ]:
arglatlon(50.0,210.0,tlat,tlon) # -> returns array[jind,iind]